In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 6.1MB/s 
     |████████████████████████████████| 890kB 20.0MB/s 
     |████████████████████████████████| 1.0MB 39.0MB/s 
     |████████████████████████████████| 3.7MB 23.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e1d03013c167cce9eb3a2bc426247ceba63310aab0b3eb48facc8c4bca65a1ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
!pip install wandb

     |████████████████████████████████| 1.4MB 6.2MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 112kB 45.7MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 460kB 39.0MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=09b13cbdc747ade9d3bd912e411a309a0bd4d9ae3b7a89bf5123d417f5d99f63
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=83cd07435269194036391d109471ef6c61980ef2aa67e68d0b5f8510113f197d
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=49

In [0]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 0f899bbf33dae82e8872c7b94e506e9e25177178
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import wandb
wandb.init(project="bert_ita_train")

W&B Run: https://app.wandb.ai/flaviodipalo/bert_ita_train/runs/1zorxk3v

In [0]:
%%writefile utils.py 

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" BERT classification fine-tuning: utilities to work with GLUE tasks """

from __future__ import absolute_import, division, print_function

import csv
import logging
import os
import sys
from io import open

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score

from multiprocessing import Pool, cpu_count
from tqdm import tqdm

logger = logging.getLogger(__name__)
csv.field_size_limit(2147483647)

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None,lang_id=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
        self.lang_id = lang_id



class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id,lang_id=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.lang_id = lang_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            #lang_id = line[4]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label, lang_id=None))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True):

    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    #lang_ids = [int(example.lang_id)]*max_seq_length
    #assert len(lang_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id,
                        lang_id = None)
    
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True,
                                 process_count=cpu_count() - 2):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}
    
    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id) for example in examples]

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

GLUE_TASKS_NUM_LABELS = {
    "binary": 2
}


Writing utils.py


In [0]:
!mkdir data

from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, 
                                  DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer)

from transformers import AdamW, get_linear_schedule_with_warmup

from utils import (convert_examples_to_features,
                        output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



args = {
    'data_dir': 'data/',
    'model_type': 'distilbert',
    'model_name':'distilbert-base-uncased',
    #'model_name':'distilbert-base-multilingual-cased',
    #'model_type': 'xlm',
    #'model_name':'xlm-mlm-17-1280',
    
    'task_name': 'binary',
    'output_dir': 'outputs/',
    'cache_dir': 'cache/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    #'output_mode': 'classification',
    'output_mode': 'regression',
    'train_batch_size': 4,
    'eval_batch_size': 4,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 45,
    'weight_decay': 0,
    'learning_rate': 4e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 2000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': True,
    'reprocess_input_data': True,
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
with open('args.json', 'w') as f:
    json.dump(args, f)

In [0]:
if os.path.exists(args['output_dir']) and os.listdir(args['output_dir']) and args['do_train'] and not args['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args['output_dir']))

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer), 
    'distilbert': (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [0]:
config = config_class.from_pretrained(args['model_name'], num_labels=1, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])
#config = config_class(dropout=0.1, attention_dropout=0.1,seq_classif_dropout=0.2,num_labels=1)
#tokenizer = tokenizer_class()

INFO:filelock:Lock 139992443848968 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpn52gp9tc


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json in cache at /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
INFO:filelock:Lock 139992443848968 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e

INFO:filelock:Lock 139992443847344 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp0g69gj4z


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:filelock:Lock 139992443847344 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f7

In [0]:
ENGLISH = False ## used to chose between the english and italian dataset. 
GENERATE_TRANSFER_LEARNING = False ## used to generate a dataframe based only on. 
ONE_SHOT = False ## used to predict only based on one shot scenarios. 
FINE_TUNING_ONLY = False ## only trains the classifier added on bert hidden neurons. 

In [0]:
task = args['task_name']
processor = processors[task]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
def load_and_cache_examples(task, tokenizer, evaluate=False):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir']) if evaluate else processor.get_train_examples(args['data_dir'])
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

    #all_lang_ids = torch.tensor([f.lang_id for f in features], dtype=torch.long)
    
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)#all_lang_ids)
    return dataset

In [0]:
from torch.utils.data import SubsetRandomSampler
import numpy as np 
def train(train_dataset, model, tokenizer,validation_split=None, shuffle_dataset=True):
    #train_sampler = RandomSampler(train_dataset)
    #train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])

    dataset_size = len(train_dataset)
    indices = list(range(dataset_size))
    
    if shuffle_dataset :
      np.random.seed(100495)
      np.random.shuffle(indices)
    if validation_split is not None: 
      split = int(np.floor(validation_split * dataset_size))
      train_indices, val_indices = indices[split:], indices[:split]
      train_sampler = SubsetRandomSampler(train_indices)
      valid_sampler = SubsetRandomSampler(val_indices)
      valid_dataloader = DataLoader(train_dataset, sampler=valid_sampler, batch_size=args['train_batch_size'])
    else: 
      train_sampler = RandomSampler(train_dataset)

    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])

    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps = t_total)
    
    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)
    logger.info(" TIPO DEL DATASET: %s", type(train_dataset))

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0

    if FINE_TUNING_ONLY: 
      for name, param in model.named_parameters():
        if 'distilbert' in name: 
          param.requires_grad = False
    
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
  # italian language id. 

    # We reshape it to be of size (batch_size, sequence_length)
    for _ in train_iterator:
        
        ##TODO: loggare epoca e loss alla fine dell'epoca wandb.log({'epochs':global_step, 'loss':tr_loss / global_step})

        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        valid_loss_list = [] 
        train_loss_list = []
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      #'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3],
                      #'langs': batch[4] ## aggiunto per XLM 
                      }
            
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            train_loss_list.append(loss.item())
            print("\r%f" % loss, end='')

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])
           
            train_loss_early = loss.item()
            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                    # Log metrics
                    if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(model, tokenizer)

                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)

        #### EARLY STOPPING PART 
        # clear lists to track next epoch
        if validation_split is not None: 


          model.eval()
          eval_loss = 0.0
          tmp_eval_loss = 0.0
          nb_eval_steps = 0.0
          preds = None
          out_label_ids = None
          
          for batch in tqdm_notebook(valid_dataloader, desc="Validation"):
            batch = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[3]}
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]
                valid_loss_list.append(tmp_eval_loss.item())
        #print(train_loss_list)
        #print(valid_loss_list)

          wandb.log({'train_loss':np.mean(train_loss_list), 'valid_loss':np.mean(valid_loss_list)})
        else: 
          wandb.log({'train_loss':np.mean(train_loss_list)})
        """
              if preds is None:
                  preds = logits.detach().cpu().numpy()
                  out_label_ids = inputs['labels'].detach().cpu().numpy()
              else:
                  preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                  out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

          eval_loss = eval_loss / nb_eval_steps
          if args['output_mode'] == "classification":
              preds = np.argmax(preds, axis=1)
          
          
          # print training/validation statistics 
          # calculate average loss over an epoch
          train_loss = np.average(train_losses)
          valid_loss = np.average(tmp_eval_loss)
          avg_train_losses.append(train_loss)
          avg_valid_losses.append(tmp_eval_loss)
          
          logger.info('[EARLY STOPPING INFO], training loss: %s, validation loss: %s', train_loss, validation_loss)
          # early_stopping needs the validation loss to check if it has decresed, 
          # and if it has, it will make a checkpoint of the current model
          
          early_stopping(valid_loss, model)
          
          if early_stopping.early_stop:
              logger.info('EARLY STOPPING BREAKING CYCLE')
              return global_step, tr_loss / global_step
          """


    return global_step, tr_loss / global_step

In [0]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, matthews_corrcoef, confusion_matrix, accuracy_score, f1_score, r2_score
from scipy.stats import pearsonr
def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):

    if args['output_mode'] == 'regression': 
      rmse = np.sqrt(mean_squared_error(labels,preds))
      r_2 = r2_score(labels, preds)
      mae = mean_absolute_error(labels,preds)
      return {
          "rmse": rmse,
          "r_2": r_2,
          "mae": mae

      }, get_mismatched(labels, preds)


    else: 
      mcc = matthews_corrcoef(labels, preds)
      tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
      accuracy = accuracy_score(labels, preds)
      f1 = f1_score(labels, preds)

      return {
          "mcc": mcc,
          "tp": tp,
          "tn": tn,
          "fp": fp,
          "fn": fn, 
          "accuracy": accuracy, 
          "f1": f1, 
      }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels, ):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def evaluate(model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, evaluate=True)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
        print(preds)
    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results, wrong

In [0]:
##DATA LOADING 

import pandas as pd
from tqdm import tqdm_notebook
import numpy as np 
df = pd.read_pickle('/content/drive/My Drive/ADReSS/train/transcriptions.pickle')
pd.set_option('display.max_rows', 300)
print(list(df))


## shuffle the dataset 
#df = df.sample(frac=1, random_state=100495).reset_index(drop=True)

df = df[df.mmse != ' NA']
df = df[df.text != ' None']
df.dropna(inplace=True)

df_cookie = df 
df_cookie.head(200)

"""
prefix = 'data/'
df = pd.read_pickle('/content/drive/My Drive/Thesis/interview_all_tasks_gra_mor_par_stringtext_mmse.pickle')
df = df[df.task.isin(['cookie'])]
df_en = df.sample(frac=1, random_state=100495).reset_index(drop=True)

df = pd.read_pickle('/content/drive/My Drive/Thesis/italian_patients_152_translated.pickle')
df = df.sample(frac=1, random_state=100495).reset_index(drop=True)

df['label'] = df.apply(lambda row: int(row.MMSE <24), axis = 1) 
df_cookie = df 
print(list(df_cookie))
"""
print(df_cookie)


['ID   ', ' age', ' gender ', 'mmse', 'label', 'text']
     ID      age  ... label                                               text
0    S079     59  ...     1  mhm . there 's a young boy & uh going in a coo...
1    S080     55  ...     1  okay he 's fallin ( g ) off a chair [ : stool ...
2    S081     69  ...     1  < well the kid > [ // ] the girl 's laughin ( ...
3    S082     66  ...     1  & =clears : throat well & =clears : throat & u...
5    S084     68  ...     1  alright . [ + ] I see the little boy stealing ...
6    S086     57  ...     1  there 's a little girl . and a little boy stan...
7    S087     70  ...     1  & hm < a lady > [ / ] a lady and her children ...
8    S089     65  ...     1  so she will find her . and xxx the mother wash...
9    S090     59  ...     1  mhm . [ + ] oh I see a part of the whole kitch...
10   S092     74  ...     1  the little boy is up on [ / ] & uh on this thi...
11   S093     68  ...     1  & uh well the children are climbing up and he .

In [0]:
df_real_test = pd.read_pickle('/content/drive/My Drive/ADReSS/test/transcriptions.pickle')
print(list(df_real_test))

['ID', 'age', 'gender', 'mmse', 'label', 'text']


In [0]:
prediction_list = [27.317049,29.739937,28.880442,28.009893,26.9452, 10.853231
,24.062252,23.663597,25.826572,28.419922,23.670488,12.084503
,29.751371,24.49347,28.341967,23.590767,14.9259405, 29.708874
,29.746342,19.262901,29.748056,18.711681,14.511718,29.651728
,29.440687,21.529028,29.526741,17.706923,17.335941,22.870455
,11.807484,15.171249,9.462222,29.52437,23.633614,28.913319
,29.087828,24.971613,25.777164,29.667665,25.995598,29.544582
,22.855505,20.03634,28.33276,27.76795,29.596169,29.691006]
prediction_list = np.array(prediction_list)
original_list = np.array(df_real_test.mmse)
diff = prediction_list - original_list

#print(diff)
#mae = [] 
#for element in diff: 
#  mae.append(abs(element))
#np.mean(mae)

for index, row in df_real_test.iterrows():
    print('{};{}'.format(row['ID'].split()[0], prediction_list[index]) )

S160;27.317049
S161;29.739937
S162;28.880442
S163;28.009893
S164;26.9452
S165;10.853231
S166;24.062252
S167;23.663597
S168;25.826572
S169;28.419922
S170;23.670488
S171;12.084503
S172;29.751371
S173;24.49347
S174;28.341967
S175;23.590767
S176;14.9259405
S177;29.708874
S178;29.746342
S179;19.262901
S180;29.748056
S181;18.711681
S182;14.511718
S183;29.651728
S184;29.440687
S185;21.529028
S186;29.526741
S187;17.706923
S188;17.335941
S189;22.870455
S190;11.807484
S191;15.171249
S192;9.462222
S193;29.52437
S194;23.633614
S195;28.913319
S196;29.087828
S197;24.971613
S198;25.777164
S199;29.667665
S200;25.995598
S201;29.544582
S202;22.855505
S203;20.03634
S204;28.33276
S205;27.76795
S206;29.596169
S207;29.691006


In [0]:
mmse_list = []
for mmse in df_cookie.mmse: 
  mmse_list.append(int(mmse))
print(np.mean(mmse_list))

mean_list = [np.mean(mmse_list)]*len(mmse_list)

from sklearn.metrics import mean_squared_error,mean_absolute_error
print(np.sqrt(mean_squared_error(mmse_list,mean_list)))
print(mean_absolute_error(mmse_list,mean_list))
import seaborn as sns, numpy as np
sns.set(); np.random.seed(0)
x = np.random.randn(100)
ax = sns.distplot(mmse_list,bins=30)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
REAL_TEST = True   

from collections import defaultdict
dict_list = defaultdict(list)

n_folds = 5
test_percentage = 1/n_folds
df_len = len(df)

for i in range(n_folds): 
  !rm -r outputs
  !mkdir outputs 
  ##TODO recupera cose. 
  number_of_samples = int(df_len*test_percentage)

  msk = np.random.rand(len(df)) > 0


  if not GENERATE_TRANSFER_LEARNING: 
    msk[i*number_of_samples:(i+1)*number_of_samples] = False

  #msk2 = np.random.rand(len(df)) > 0.7
  #msk = np.array(msk) & np.array(msk2)
  df_cookie.sample(frac=1, random_state=1).reset_index(drop=True)
  if not REAL_TEST: 
    train_ = df_cookie[msk]
    test_ = df_cookie[~msk]

    train_df = pd.DataFrame({
        'id':range(len(train_)),
        'label':train_.mmse,
        'alpha':['a']*train_.shape[0],
        'text': train_.text,
        #'lang': int(tokenizer.lang2id['en'])
    })
    
    #dev_df = pd.DataFrame({
    #    'id':range(len(test_)),
    #    'label':test_.mmse,
    #    'alpha':['a']*test_.shape[0],
    #    'text': test_.text,
        #'lang': int(tokenizer.lang2id['it'])
    #})

  else: 
    train_df = pd.DataFrame({
        'id':range(len(df_cookie)),
        'label':df_cookie.mmse,
        'alpha':['a']*df_cookie.shape[0],
        'text': df_cookie.text,
        #'lang': int(tokenizer.lang2id['en'])
    })

    dev_df = pd.DataFrame({
        'id':range(len(df_real_test)),
        'label':df_real_test.mmse,
        'alpha':['a']*df_real_test.shape[0],
        'text': df_real_test.text,
        #'lang': int(tokenizer.lang2id['it'])
    })


  dev_df.to_csv('data/dev.tsv', sep='\t', index=False, header=False, columns=dev_df.columns)
  train_df.to_csv('data/train.tsv', sep='\t', index=False, header=False, columns=train_df.columns)

  if args['do_train']:
    train_dataset = load_and_cache_examples(task, tokenizer)
    
    ## chose a pretrained model 
    #model = model_class.from_pretrained(args['model_name'])
    
    ## the model is not pretrained 
    model = model_class(config)

    model.to(device);
    if not ONE_SHOT: 
      global_step, tr_loss = train(train_dataset, model, tokenizer)
      logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

  if args['do_train']:
    if not os.path.exists(args['output_dir']):
            os.makedirs(args['output_dir'])
    logger.info("Saving model checkpoint to %s", args['output_dir'])
    
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    
    if GENERATE_TRANSFER_LEARNING: 
      model_to_save.save_pretrained('./drive/My Drive/Thesis/xlm_english_trained/')
      tokenizer.save_pretrained('./drive/My Drive/Thesis/xlm_english_trained/')
    else:
      model_to_save.save_pretrained(args['output_dir'])
      tokenizer.save_pretrained(args['output_dir'])
      
    torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))
    results = {}
    if GENERATE_TRANSFER_LEARNING: 
      logger.info('STOPPING TRAINING, DATASET GENERATED')
      break 

    if args['do_eval']:
        checkpoints = [args['output_dir']]
        if args['eval_all_checkpoints']:
            checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args['output_dir'] + '/**/' + WEIGHTS_NAME, recursive=True)))
            logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class.from_pretrained(checkpoint)
            model.to(device)
            result, wrong_preds = evaluate(model, tokenizer, prefix=global_step)
            result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
            results.update(result)

            for k, v in results.items():
              try: 
                dict_list[k].append(v)
              except Exception as e: 
                print(e)
  if REAL_TEST: 
    break 


In [0]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')



list_files = [27.317049,  29.739937 , 28.880442 , 28.009893,  26.9452  ,  10.853231,
 24.062252,  23.663597 , 25.826572  28.419922  23.670488  12.084503
 29.751371  24.49347   28.341967  23.590767  14.9259405 29.708874
 29.746342  19.262901  29.748056  18.711681  14.511718  29.651728
 29.440687  21.529028  29.526741  17.706923  17.335941  22.870455
 11.807484  15.171249   9.462222  29.52437   23.633614  28.913319
 29.087828  24.971613  25.777164  29.667665  25.995598  29.544582
 22.855505  20.03634   28.33276   27.76795   29.596169  29.691006 ]

In [0]:
print(dict_list)
for k in dict_list:
  print(k, np.mean(dict_list[k]))